# STOR 601: C++ assignment

*Graham Burgess - March 2023*

In [1]:
#include <iostream>
#include <vector>
#include <limits>
#include <stdlib.h> 
#include <algorithm>
#include <fstream>
#include <sstream>
#include <string>
#include <cmath>

## Part 1: "anom" function

The **anom** function is a member function of the **Line_Segment** structure, which is formalised below, along with the Point structure. 

In [2]:
struct Point
{
    double x;
    double y;
    Point(double _x = 0.0, double _y = 0.0)
    {
        x = _x;
        y = _y;   
    }
    Point operator-(Point p)
    {
       return Point(x - p.x,y - p.y);
    }
};

struct Line_Segment
{
    Point a;
    Point b;
    Line_Segment(Point _a, Point _b)
    {
        a = _a;
        b = _b;
    }
    double anon(Line_Segment other)
    {
        Point c = b - a;
        Point d = other.b - other.a;
        return c.x * d.y - d.x*c.y;
    }
};

**How is the anom function defined?**

The anom function is a member function, available to instances of the Line_Segment structure. Let us call such an instance *one*. The anom function takes another separate instance of the Line_Segment structure as an argument. This instance is defined as *other* within the function definition above. The output of the function is a real number. Therefore we can think of anon as being a function of two line segments (*one* and *other*) which returns a real number. 

**What does the anom function do with the two line segments *one* and *other*?**

Firstly, the anom function characterises each original line segment (*one* and *other*) with a new point in the x-y plane. New line segments which are formed by the origin and these new points are effectively the same as the original line segments but transformed so that they start at the origin but have the same magnitude and direction as the original line segments. By transforming both the line segments *one* and *other*, this step has effectively normalised the line segments so they both start at the origin. This normalisation has implicitly interpretted the line segments as vectors with a direction, assuming that they begin at their first defined point and end at their second. 

Secondly, the anom function returns a cross product of the two new points which characterise the line segments. Geometrically speaking, this can be thought of as calculating the area of the parallelogram which is created using each normalised line segment as a side of the parallelogram. 

**An example**

The following diagrams illustrate these two steps using an example with the following two lines: 

* line 1 starts at $a$ and finishes at $b$ where the x- and y-coordinates of $a$ and $b$ are $(2,1)$ and $(8,3)$ respectively. 
* line 2 starts at $c$ and finishes at $d$ where the x- and y-coordinates of $c$ and $d$ are $(6,6)$ and $(7,9)$ respectively. 

The first diagram shows lines 1 and 2 in blue and red respectively. 

![lineplot1](lineplot1.png)

The next diagram shows how the first part of the function characterises the first line with a point $e$, the second line with a point $f$, and that these two points, together with the origin, normalise the two original line segments so they start at the origin, but maintain their length and direction. The normalised line segments are drawn with dashed lines in the diagram below. 

![lineplot2](lineplot2.png)

The second step of the alorigthm is to calculate the cross product of the two points $e$ and $f$. As the diagram below illustrates, this is equivalent to calculating the area (shaded) of the parallelogram created using the normalised line segments as sides of the parallelogram. An auxiliary point $aux$ is used at coordinates $(7,5)$ to complete the parallelogram. 

![lineplot3](lineplot3.png)

The area of the shaded region is equal to the cross product of $e$ and $f$. $Area = e_x f_y - e_y f_x = 18-2 = 16$. 

**Summary**

The anom function gives the cross product of two line segments which have been normalised so they both start at the origin in the x-y plane. 

## Part 2: line intersection

The following pseudocode describes an algorithm I designed which, given two line segments in the same plane, determines whether they intersect or not.

![LineIntersection](Line_Segment_Intersection.png)

## Part 3: Jarvis March implementation

The code in this part implements the Jarvis march algorithm in C++. The algorithm is almost identical to the algorithm which I provided in Python in January. The only change is: 

* the code no longer computes the angle between two line segments which start at the origin of the two-dimensional plane, instead it simply computes the cross product and dot product of the two points corresponding to the end of those two line segments. This is a slightly simpler way of determining the orientation of the two line segments. 

There are also some minor changes to the code, as detailed below: 

* there is no longer a *line* class since this was used to create instances of line segments starting at the origin and ending at another point. The information can more simply be captured with points (from the point class) corresponding to the end of those line segments. 
* there is no longer a *set_of_points* class. I have instead simply collected points into a C++ vector. This was becasue (a) the C++ vector provides what I need and there was no vector data structure in Python and (b) I built my *set_of_points* in Python by inheriting from the *tuple* class and I have not yet studied inheritance in C++ structures, which i felt was not really necessary for this task. 
* Due to the previous point, the functions *find_leftmost_point*, *find_new_point* and *find_convex_hull* are now functions in their own right, rather than member functions of the *set_of_points* class. 

The text and code below provides some information regarding C++ versions, introduces the user to the structures and functions, and finally analyses the five datasets provided for the assignment. Finally, there is a discussion of the 5 R's. 

### Information for user

This code has been developed and tested using the C++17 version of C++. Running the code chunk below verifies that you are running this on the same version.

In [3]:
if (__cplusplus == 202002) {
    std::cout << "C++20" << std::endl;
}
else if (__cplusplus == 201703) {
    std::cout << "C++17" << std::endl;
}
else if (__cplusplus == 201402) {
    std::cout << "C++14" << std::endl;
}
else if (__cplusplus == 201103) {
    std::cout << "C++14" << std::endl;
}
else if (__cplusplus == 199711) {
    std::cout << "C++14" << std::endl;
}
else {
    std::cout << "non-standard version of C++" << std::endl;
}

C++17


### Functions and structures

I now introduce the functions and structures that I have written to execute the algorithm. Each function/structure contains a docstring with helpful information. Some further commenting is included in the code cells below, for clarity. It is worth noting that the analysis is run by calling the "jarvis_march" function, which itselfs calls the other functions and structures, directly or indirectly.

In [4]:
struct point
/*
A structure to represent a single point on a two-dimensional plane

Attributes
----------
x : double
    x-coordinate
y : double
    y-coordinate

Methods
-------
None
*/    
{  
    double x;
    double y;
    
    point(double _x,double _y)
    /*
    Initialise instance of the point structure

    Parameters
    ----------
    _x : double
        x-coordinate
    _y : double
        y-coordinate

    Returns
    -------
    None
    */
        
    {
        x = _x;
        y = _y;   
    }
};

In [5]:
struct triplet_of_points
/*
A structure to represent a group of three points (one, two and three) on a two-dimensional plane

Attributes
----------
right_turn : 
    True if a traversal from point 1 to point 3 via point 2 involves a right turn.
    I.e. if the counterclockwise angle between a and b is 180 degrees or less. 
    If the traversal involves just a straight line, right_turn = True
    If the traversal involves a 360 degree turn, right_turn = False
collinearity : 
    True if the three points are collinear. 
a :
    the dimensions of the line between the first and second point in the triplet
b : 
    the dimensions of the line between the third and second point in the triplet
determinent : 
    the determinent of matrix (a^T, b^T)
    I.e. the cross product between a and b. 
dot_product : 
    the dot product of a and b

Methods
-------
find_orientation:
    determines whether a traversal from point 1 to point 3 via point 2 involves a right turn. 
*/    
{  
    bool right_turn {};
    bool collinearity {};
    double determinent {};
    double dot_product {};
    
    triplet_of_points(point p1, point p2, point p3)
    /*
    Initialise instance of the triplet_of_points class

    Parameters
    ----------
    p1 : point
        First point in triplet
    p2 : point
        Second point in triplet
    p3 : point
        Third point in triplet
    Returns
    -------
    None
    */
        
    {
        point a(p1.x - p2.x, p1.y - p2.y);
        point b(p3.x - p2.x, p3.y - p2.y);
        determinent = a.x * b.y - b.x * a.y;
        dot_product = a.x * b.x + a.y * b.y ; 
    }
    
    void find_orientation()
    /*
    Find the orientation (i.e. right-turning or not) of the triplet. 
    Also identify if points on triplet are collinear 

    Parameters
    ----------
    None
    
    Returns
    -------
    None
    */
      
    {        
        if (determinent > 0) // right turn
        {
            right_turn = true;
            collinearity = false;
        }
        else if (determinent == 0 && dot_product < 0) // straight line, categorised as right turn
        {
            right_turn = true;
            collinearity = true;
        }
        else if (determinent == 0 && dot_product > 0) // back on itselft, cateogorised as left turn
        {
            right_turn = false;
            collinearity = true;
        }
        else if (determinent < 0) // left turn
        {
            right_turn = false;
            collinearity = false;
        }
    }
};

In [6]:
int find_leftmost_point(double leftmost_val, std::vector<point> points)
/*
Finds the leftmost point in a set of points. 
If there are two points on the left with same x value, this function will choose the one it encounters first.
This is arbitrary but does not affect the outcome of the algorithm, only the ordering of points on the hull.

Parameters
----------
leftmost_val : double
    the current estimate of the x-value of the leftmost point. 
points : vector<point>
    vector of points which are being analysed

Returns
-------
leftmost_index_update : int
    index (within the points vector) of the leftmost point. 
*/  

{    
double leftmost_val_update {leftmost_val};
int leftmost_index_update {};

    for(int p = 0; p < points.size(); p++)
    {
        if (points[p].x < leftmost_val_update)
        {
            leftmost_val_update = points[p].x;
            leftmost_index_update = p;            
        }
    }
    return leftmost_index_update;
}

In [7]:
int find_new_point(std::vector<point> points, std::vector<int> exceptions)
/*
Finds the index of a new point at random from a vector of points

Parameters
----------
points : vector<point> 
    vector of points which are being analysed
exceptions : vector<int>
    A vector of indices of points not to be included when choosing a new index at random

Returns
-------
new_point : int
    Index of a new point chosen at random from the set. 
*/

{
    bool looking_for_new_point {true};
    int new_point {};
    while (looking_for_new_point == true)
    {
        new_point = rand() % points.size();
        if (std::find(exceptions.begin(), exceptions.end(), new_point) == exceptions.end()) // new point not exception
        {
            looking_for_new_point = false;
        }
    }
    return new_point;
}

In [8]:
std::vector<point> find_convex_hull(std::vector<point> points)
/*
Finds the convex hull of a vector of points. 
First this function deals with indices: i.e. it finds the indices of the points in the vector which are on the hull. 
It then translates these indicies into a vector of points which define the convex hull. 

Parameters
----------
points : vector<point>
    vector of points being analysed

Returns
-------
convex_hull_points : vector<point>
    vector of points on the convex hull
*/    

{
    // set up attributes
    std::vector<int> convex_hull {}; // list of indices indicating list-position of the points on the hull
    std::vector<point> convex_hull_points {}; // list of points (from the points class) on the convex hull
    bool all_points_collinear {true}; // change this if we find non-collinear points
    int leftmost_index;
    int rightmost_index;
    double leftmost_val {std::numeric_limits<double>::infinity()};
    
    // First deal with special cases of small sets of points
    if (points.size() == 0){
        std::cout << "No data points to analyse" << std::endl;
    }
    else if (points.size() == 1){
        std::cout << "Only one data point to analyse" << std::endl;
        convex_hull_points.push_back(points[0]);
    }
    else if (points.size() == 2){
        std::cout << "only two data points to analyse" << std::endl;
        // add leftmost point
        leftmost_index = find_leftmost_point(leftmost_val, points);
        convex_hull.push_back(leftmost_index);
        convex_hull_points.push_back(points[leftmost_index]);
        // add rightmost point
        if (leftmost_index == 0)
        {
            rightmost_index = 1;
        }
        else {
            rightmost_index = 0;
        }
        convex_hull.push_back(rightmost_index);
        convex_hull_points.push_back(points[rightmost_index]);
    }
    else {
        leftmost_index = find_leftmost_point(leftmost_val, points); // find leftmost point
        convex_hull.push_back(leftmost_index);
        bool not_complete_hull {true}; // this will change to False once the convex hull reaches its starting point
        
        // main while loop
        while (not_complete_hull == true)
        {
            // identify end of the current hull
            int end_of_hull_index {convex_hull.back()};
            
            // select candidate
            std::vector<int> exceptions {end_of_hull_index};
            int candidate {};
            candidate = find_new_point(points, exceptions);
            
            // test the candidate with test points
            for(int test_point = 0; test_point < points.size(); test_point++)
            {
                if (test_point != candidate && test_point != end_of_hull_index)
                {
                    // create new instance of triplet class from the end of the hull -> test point -> candidate
                    triplet_of_points triplet(points[end_of_hull_index], points[test_point], points[candidate]);
                    triplet.find_orientation();
                    
                    // update candidate if angle theta in the triplet from a -> b counterclockwise satisfies 0 < theta <= 180
                    if (triplet.right_turn == true)
                    {
                        candidate = test_point;
                    }
                    
                    /* update candidate if triplet is collinear, candidate on the convex hull but not test_point
                    this stops the algorithm prioritising a point already on the convex hull ...
                    ... when comparing against a collinear test point ...
                    ... It might then have chosen a suboptimal point at the next test point, but this ensures not. */
                    if (triplet.collinearity == true && std::find(convex_hull.begin(), convex_hull.end(), candidate) != convex_hull.end() && find(convex_hull.begin()+1, convex_hull.end(), test_point) == convex_hull.end())
                    {
                        candidate = test_point;
                    }
                    
                    // Update instance attribute if a non-collinear triplet has been found. 
                    if (triplet.collinearity == false)
                    {
                        all_points_collinear = false;
                    }
                }
            }
            
            // update hull
            convex_hull.push_back(candidate);

            // is the hull complete?
            if (std::find(convex_hull.begin(), convex_hull.end()-1, convex_hull.back()) != convex_hull.end()-1)
            {
                not_complete_hull = false;
                if (convex_hull.front() == convex_hull.back())
                {
                    convex_hull.pop_back();
                }
            }
        }
            
        // If not all points are collinear, then simply add all point objects the hull, based on their index. 
        if (all_points_collinear == false)
        {
            for(int point_to_add = 0; point_to_add < convex_hull.size(); point_to_add++)
            {
                convex_hull_points.push_back(points[convex_hull[point_to_add]]);
            }
        }

        /* If all the points collinear, ignore self.convex_hull and create self.convex_hull_points from scratch
        This is because in this case, the algorithm will go from the start to the furthest point to find the hull ...
        ... and then it will go back, continuing to add points onto the hull, but in this case ...
        ... it might skip some points and go directly back to the first point. 
        But we can instead generate it from scratch. */   

        else
        {
            for (int i = 0; i < convex_hull.size(); i++)
            {
                // add first point
                if (i == 0)
                {
                    convex_hull_points.push_back(points[convex_hull[i]]);
                }

                // go out to the end of the hull
                else if (std::find(convex_hull.begin(), convex_hull.begin()+(i-1), convex_hull[i]) == convex_hull.begin()+(i-1))
                {
                    convex_hull_points.push_back(points[convex_hull[i]]);
                }

                else
                {
                    break;
                }          
            }
        }
    }
    return convex_hull_points;
};

In [9]:
// function to read points: copied from the lecture notes. 

template <typename T>
T lexical_cast(const std::string& str)
{
    T var;
    std::istringstream iss;
    iss.str(str);
    iss >> var;
    // deal with any error bits that may have been set on the stream
    return var;
}

template<typename data_type,
template <typename... table_type_args> typename table_type,
template <typename... row_type_args> typename row_type>
table_type<row_type<data_type> > csvtable(const std::string& filename)
{
  table_type<row_type<data_type> > table;
  std::ifstream infile(filename);
  while(infile)
    {
      std::string s;
      if(!getline(infile,s)) break;
      std::istringstream ss(s);
      row_type<data_type> row;
      while(ss)
      {
         std::string s;
         if(!getline(ss,s,',')) break;
         row.push_back(lexical_cast<data_type>(s)); 
      }
      table.push_back(row);
    }
  return table;
}

#include <list>
#include <vector>

#define readcsv csvtable<double,std::list,std::vector>

In [10]:
void jarvis_march(std::string filename)
/*
Implement the Jarvis march algorithm. 

This function reads the data from the given CSV, initialises a vector of points and finds its convex hull. 
Outputs are printed from within this function, as opposed to being returned from the fuction. 

Parameters
----------
filename : string
    Name of CSV file containing (x,y) point information in two columns. This should include '.csv'

Returns
-------
None
*/
    
{
    // read points
    std::list<std::vector<double> > xy_pairs {readcsv(filename)};
    std::vector<point> points {};
    for(std::vector<double>& xy_pair : xy_pairs)
    {
        point new_point(xy_pair[0],xy_pair[1]);
        points.push_back(new_point);
    }

    // find hull
    srand(10);
    std::vector<point> hull {};
    hull = find_convex_hull(points);
    
    // print
    std::cout << "-----------------" << std::endl;
    std::cout << "Analysis of data in " << filename << std::endl;
    std::cout << "There are " << points.size() << " points in the set and " << hull.size() << " points on the convex hull." << std::endl;
    std::cout << "The points on the hull have the following (x,y) co-ordinates:" << std::endl;
    for(int hull_index = 0; hull_index < hull.size(); hull_index++)
    {
        std::cout << "(" << hull[hull_index].x << "," << hull[hull_index].y << ")" << std::endl;
    }
}

In [ ]:
std::vector<std::vector<double>> jarvis_march_alt(std::vector<double> x, std::vector<double> y)
/*
Implement an alternative Jarvis march algorithm (for R package build). 

This function takes the inputted x and y vectors, initialises a vector of points and finds its convex hull. 
a vector with x and y coords of convex hull are returned. 

Parameters
----------
x : std::vector<double>
    x coords
y : std::vector<double>
    y coords

Returns
-------
hull : std::vector<std::vector<double>>
    x and y coords of hull
*/
{
    // read points
    // std::list<std::vector<double> > xy_pairs {readcsv(filename)};
    std::vector<point> points {};
    for(int i = 0; i < x.size(); i++)
    {
        point new_point(x[i],y[i]);
        points.push_back(new_point);
    }

    // find hull
    srand(10);
    std::vector<point> hull_points {};
    hull_points = find_convex_hull(points);
    
    // output
    std::vector<double> hull_x {};
    std::vector<double> hull_y {};
    for(int hull_index = 0; hull_index < hull_points.size(); hull_index++)
    {
        hull_x.push_back(hull_points[hull_index].x);
        hull_y.push_back(hull_points[hull_index].y);
    }
    std::vector<std::vector<double>> hull {};
    hull.push_back(hull_x);
    hull.push_back(hull_y);
    return hull;
};

In [ ]:
//std::vector<double> jarvis_march_alt(std::vector<double> x, std::vector<double>& y)

std::vector<double> x_trial {1,1,1.5,1.5,2,2};
std::vector<double> y_trial {1,2,1.5,2.5,2,1};

std::vector<double> hull_alt_x {};
hull_alt = jarvis_march_alt(x_trial, y_trial);

### Analysis

The output below illustrates the results of implementing the algorithm on the five datasets provided. In the Python version, graphs were used to verify the results by eye. C++ is not suited to producing graphs, but I have shown the python plots from assignment one below the C++ output for four of the datasets (all except line.csv, which is trivial). Further to a visual check with the python charts, a spotcheck of the coordinates of the points on each convex hull has been performed between Python and C++, to ensure the results are consistent.

In [ ]:
// vector of datasets
std::vector<std::string> datasets {"random.csv", "square.csv", "line.csv", "pgram.csv", "pines.csv"};

// analyse each one
for (int dataset = 0; dataset < datasets.size(); dataset++)
{
    jarvis_march(datasets[dataset]);
}

![ConvexHulls](convex_hulls_python.png)

### The 5 R's

Given that the algorithm is almost identical and the coding only has minor structural changes (aside, of course, from it being in a different language), I feel that my C++ version of this algorithm performs similarly to my Python version, with respect to the five R's. In particular - I have retained a check on the version of C++ used (**re-runnable**), I have continued to control the random number stream (**repeatable**), I have not added anything further in terms of **reproducibility** (although I will enhance this aspect when packaging my algorithm and making it availabe on GitHub in the next assignment), I have provided equally thorough documentation and have avoided hardcoding to make the code **reusable** and I have not changed the pseudocode so my results should remain as **replicable** as before. 

It is worth noting that since the code is written in C++, which is a less intuitive language compared to Python, it would be less easy for someone else to **reuse** my code simply because it has harder to read off a C++ script and know what is going on - there is stronger reliance on the documentation in this respect. 

## Part 4: Largest distance between points

I have designed a brute-force algorithm which, given a set of at least two points in a plane, outputs the two points that are furthest apart. The following pseudo-code describes my algorithm

![Distance](Furthest_Distance_Brute_Force.png)

### Computational complexity

When there are $N$ points in the set, the number of distances which must be calculated (and compared with the current largest distance) is $\frac{N^2-N}{2}$ and so we can describe the computational complexity as $O(N^2)$. Since this is a brute-force algorithm (i.e. every distance must be checked) the number $N$ fully determins the number of distances which must be calculated and checked, so there is no difference between the best and worst case complexity in this algorithm, both are simply $O(N^2)$. 

## Part 5: Implementing and improving the algorithm

The code below implements my brute force algorithm in C++. I draw upon the same *point* structure which I defined in the Jarvis March algorithm in part 3 and I also use the same function defined in part 3 for reading in the data from CSV files. 

In [ ]:
void longest_distance(std::string filename)
/*
Implement the brute-force algorithm for finding the longest distance between two points, given a set of points. 

This function reads the data from the given CSV, initialises a vector of points and finds the furthest distance. 
Outputs are printed from within this function, as opposed to being returned from the fuction. 

Parameters
----------
filename : string
    Name of CSV file containing (x,y) point information in two columns. This should include '.csv'

Returns
-------
None
*/
    
{
    // read points
    std::list<std::vector<double> > xy_pairs {readcsv(filename)};
    std::vector<point> points {};
    for(std::vector<double>& xy_pair : xy_pairs)
    {
        point new_point(xy_pair[0],xy_pair[1]);
        points.push_back(new_point);
    }

    // find longest distance
    std::vector<std::vector<point>> furthest_pairs {};
    double furthest_distance {0};
    
    for(int i=0; i<points.size(); i++)
    {
        for(int j=i+1; j<points.size(); j++)
        {
            double dist {};
            dist = sqrt(pow(points[j].x - points[i].x, 2) + pow(points[j].y - points[i].y, 2));
            if (dist > furthest_distance)
            {
                furthest_distance = dist;
                std::vector<point> furthest_pair {};
                furthest_pair = {points[i],points[j]};
                furthest_pairs = {furthest_pair};
            }
            else if (dist == furthest_distance)
            {
                std::vector<point> furthest_pair {};
                furthest_pair = {points[i],points[j]};
                furthest_pairs.push_back(furthest_pair);
            }
        }
    }
    
    // print
    std::cout << "------------------------------------------------" << std::endl;
    std::cout << "Analysis of data in " << filename << std::endl;
    std::cout << "There are " << points.size() << " points in the set."  << std::endl;
    std::cout << "The two points which are furthest away have the following (x,y) co-ordinates:" << std::endl;
    for(int p_index = 0; p_index < furthest_pairs[0].size(); p_index++)
    {
        std::cout << "(" << furthest_pairs[0][p_index].x << "," << furthest_pairs[0][p_index].y << ")" << std::endl;
    }
    std::cout << "These two points are " << furthest_distance << " units apart." << std::endl;
    if(furthest_pairs.size() > 1)
    {
        std::cout << "Note - there are multiple pairs of points which are the furthest apart in this case" << std::endl;
        std::cout << "The other pairs have the following (x,y) coordinates" << std::endl;
        for(int pairs = 1; pairs < furthest_pairs.size(); pairs++)
        {
            std::cout << "------------" << std::endl;
            for(int p_extra = 0; p_extra < furthest_pairs[pairs].size(); p_extra++)
            {
                std::cout << "(" << furthest_pairs[pairs][p_extra].x << "," << furthest_pairs[pairs][p_extra].y << ")" << std::endl;
            }
        }
    }
}

### Analysis

The output below illustrates the results of implementing the algorithm on the five datasets provided. A visual check with the python charts (illustrated in Part 3 of this coursework) confirms that the algorithm is working as expected. 

In [ ]:
// analyse each dataset
for (int dataset = 0; dataset < datasets.size(); dataset++)
{
    longest_distance(datasets[dataset]);
}

### Improving the algorithm

The next question is - can we improve the brute force algorithm in terms of best-case computational complexity? The answer is yes, we can. To show this, we draw upon the fact that the two points in a set $\mathcal{Q}$ which are furthest apart must lie on the convex hull of $\mathcal{Q}$. Let us call these two points $a$ and $b$ and the distance between them $x$. Let us first assume that point $a$ is on the convex hull. If point $b$ was *not* on the convex hull, then there would be another point $c$, on the convex hull, with a distance from $a$ of $y$ with $y > x$. But if $y > x$ then $a$ and $b$ are not the points furthest apart in the set, and this does not make sense. It therefore cannot be true that either $a$ or $b$ are not on the convex hull if they are the furthest apart in the set $\mathcal{Q}$. 

So, an algorithm with better best-case computational complexity than the brute force algorithm would first identify points on the convex hull, and then apply the brute force algorithm only to those points on that convex hull. I will call this algorithm the improved algorithm. 

If $N >> H$ where $H$ is the number of points on the hull, then this is clearly better than our brute force algorithm. Finding the convex hull (using my algorithm from assignment 1) is $O(NH)$, and finding the furthest points on that hull is $O(H^2)$ so the combination is $O(NH)$ since this is the dominant term. This is better than $O(N^2)$ from the brute force algorithm. 

Of course, the worst case in my improved algorithm is worse than the brute-force algorithm. If all the points are on the convex hull, i.e. if $N == H$, then the second part of my improved algorithm is identical to the original brute force algorithm, but the improved algorithm adds an initial step of finding the convex hull, which clearly adds more computational time. 

My improved algorithm is described formally in the pseudo code below, making reference to my algorithm for finding the convex hull (see assignment 1 in Python) and my brute-force algorithm. 

![Distance_improved](Furthest_Distance_Improved.png)